# DLTHON

## DKTC (Dataset of Korean Threatening Conversations)

- 텍스트 다중분류 프로젝트

## 데이터셋 정보

train.csv

    1. idx = 인덱스
    2. class = 0~4
        class 0; 협박 대화
        class 1; 갈취 대화
        class 2; 직장 내 괴롭힘 대화
        class 3; 기타 괴롭힘 대화
    3. conversation = \n으로 구분된 멀티턴 텍스트 대화

test.json

    1. t_### = 인덱스
    2. text = 대화

submission.csv

    1. file_name = 인덱스
    2. class = 예측값

## 평가기준 
> - 데이터 EDA와 데이터 전처리가 적절하게 이뤄졌는가?
> - Task에 알맞게 적절한 모델을 찾아보고 선정했는가?
> - 성능향상을 위해 논리적으로 접근했는가?
> - 결과 도출을 위해 여러가지 시도를 진행했는가?
> - 도출된 결론에 충분한 설득력이 있는가?
> - 적절한 metric을 설정하고 그 사용 근거 및 결과를 분석하였는가?
> - 발표가 매끄럽게 진행되었고 발표시간을 준수하였는지? (발표 10분-15분)

## TO-DO-LIST
- 일반 대화 데이터셋 만들어야함 (800-1000개정도)
- ppt 제작
- 평가지표 : f1-score

## EDA


In [118]:
import matplotlib.pyplot as plt
import seaborn as sns

In [119]:
import os
import pandas as pd

train_data_path ="./data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [120]:
train_data.shape

(3950, 3)

## 데이터 전처리

In [121]:
!pip install soynlp

In [122]:
import re
# import emoji
from soynlp.normalizer import repeat_normalize

In [123]:
def clean(x):
    # emojis = ''.join(emoji.UNICODE_EMOJI.keys())
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣]+')
    url_pattern = re.compile(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [124]:
train_data['cleaned'] = train_data.conversation.apply(lambda x: clean(x))

In [125]:
train_data['cleaned']

0       지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼...
1       길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...
2       너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어. 그만해. 니들 놀리는거 재미없...
3       어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈...
4       저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
                              ...                        
3945    준하야 넌 대가리가 왜이렇게 크냐? 내 머리가 뭐. 밥먹으면 대가리만 크냐 너는? ...
3946    내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내 예.? ...
3947    나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지. 왜 그래. 들리겠어....
3948    누구맘대로 여기서 장사하래? 이게 무슨일입니까? 남의 구역에서 장사하려면 자릿세를 ...
3949    희정씨 네? 주말에 시간이 넘쳐나나봐 갑자기 왜그러세요? 손이 빤짝빤짝 네일했니? ...
Name: cleaned, Length: 3950, dtype: object

### 한국어 문장 분리기

In [126]:
# !pip install kss

In [127]:
samples = train_data.sample(5)

In [128]:
samples

,idx,class,conversation,cleaned
1186,1186,갈취 대화,야 꼬마야 돈 좀있냐\n무서운 어른은 피하라고 엄마가 그랬는데\n우리 무서운 어른 ...,야 꼬마야 돈 좀있냐 무서운 어른은 피하라고 엄마가 그랬는데 우리 무서운 어른 아닌...
2797,2797,직장 내 괴롭힘 대화,저기 신입은 남자친구있어?\n아 없는데요\n그럼 저기 과장님어때?\n네? 저보다 2...,저기 신입은 남자친구있어? 아 없는데요 그럼 저기 과장님어때? 네? 저보다 20살 ...
501,501,직장 내 괴롭힘 대화,진짜 별로다 김대리\n네?\n내가 그렇게 말했는데도 아직도 모르겠어?\n네?무슨 말...,진짜 별로다 김대리 네? 내가 그렇게 말했는데도 아직도 모르겠어? 네?무슨 말씀이신...
585,585,기타 괴롭힘 대화,친구야 지방대 다닐만하냐?\n 응. 편하고 좋아\n 너네학교 애들은 공부안하고 술만...,친구야 지방대 다닐만하냐? 응. 편하고 좋아 너네학교 애들은 공부안하고 술만 먹지?...
1395,1395,협박 대화,난 널 사랑해\n난 이제 너가 너무 무서워\n무슨소리야? 이쁜 입으로 이쁜말만해 입...,난 널 사랑해 난 이제 너가 너무 무서워 무슨소리야? 이쁜 입으로 이쁜말만해 입을 ...


In [129]:
samples.cleaned.iloc[0]

'야 꼬마야 돈 좀있냐 무서운 어른은 피하라고 엄마가 그랬는데 우리 무서운 어른 아닌데 그냥 돈이 필요한 어른인데 저 돈 없어요 그럼 너 휴대전화라도 줘 네? 돈이 필요하다면서 휴대전화는 왜요? 그야 너 핸드폰이 돈이 돼니까 그렇지 안돼요 왜 그러세요 코피 터지고 나서 줄래 그냥 줄래 죄송해요 한번만 봐주세요 넌 잘못한게 없어 다만 핸드폰을 안주면 큰일이 생길거야'

In [130]:
# from kss import split_sentences

# split_sentences(samples.cleaned.iloc[0])

In [131]:
# split_sentences(samples.cleaned.iloc[2])

In [132]:
# split_sentences(samples.cleaned.iloc[-1])

## BERT

In [133]:
!pip install -U transformers

In [134]:
from transformers import TFElectraModel, ElectraTokenizer

# model = TFElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator", from_pt=True)
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [135]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder를 객체로 생성
encoder = LabelEncoder()

# fit, transform 메소드를 통한 레이블 인코딩
encoder.fit(train_data['class'])

train_data["label"] = encoder.transform(train_data['class'])

In [136]:
dataset = train_data[['cleaned', 'label']]

In [137]:
dataset.sample(n=10)

,cleaned,label
2072,저기 인테리어 맞는가? 네 맞는데요 혹시 누구신지 말씀해주실 수 있나요? 정보에 입...,0
470,아 정글 씨발새끼 정글에 엄마 묻어놨냐? 계속 찾아다니네. 그니까 쉬발 ? 지가 갱...,1
3393,중구가 시키드나? 경찰들 논간에 놀아나는 거여 어이 정청 이 짱개새꺄 우리가 개 호...,3
3533,야 머하냐? 학원가려고. 돈 좀 있냐? 아니 없어. 야 니가 머가 없어 니 손에 든...,3
1563,어이 김대리 김대리가 내일부터 내것 까지 도시락 반찬 좀 싸와 네? 저희는 저희끼리...,2
2989,야이자식너 학교에서 시험뭐이따위로 점수받아와 죄송해요아빠 너 배에 물좀차봐야겠네 골...,3
603,야 돼지.지나간다 나말이야? 돼지가 말도 하네 왜 그래 너 뚱뚱한거 맞잖아 내가 뚱...,1
2944,성오야 미안한데 나 3만원만 빌려주면 안 돼? 미안해.나도 이번달에 지출이 많았어서...,0
2253,말같지도 않은 소리하네. 잘 알지도 못하면서 팁이라고 영상 올린거냐? 제 주관이고 ...,3
613,내가 정보 다 가지고 있으니 긴말 안할테니까 돈 인 당 만원씩 내놔 네? 그게 뭔소...,0


In [138]:
max_seq_len = 0
len_64_128 = 0
len_128_256 = 0

mylist = [0 for i in range(10000)]
for index, value in dataset['cleaned'].items():
    mylist[len(value)] += 1
    max_seq_len = max(max_seq_len, len(value))
    
    if len(value) >= 64 and len(value) < 128:
        len_64_128 += 1
    elif len(value) >= 128 and len(value) < 256:
        len_128_256 += 1

print(f'가장 긴 문장 길이: {max_seq_len}')
print(f'64과 128 사이 길이 문장 개수: {len_64_128}')
print(f'128과 256 사이 길이 문장 개수: {len_128_256}')

가장 긴 문장 길이: 874
64과 128 사이 길이 문장 개수: 462
128과 256 사이 길이 문장 개수: 2344


In [139]:
max_seq_len = 800

In [140]:
# !pip install -U scikit-learn

  Using cached scikit_learn-1.3.2-cp38-cp38-macosx_12_0_arm64.whl (9.4 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.1
    Uninstalling scikit-learn-1.3.1:
      Successfully uninstalled scikit-learn-1.3.1


In [144]:
from sklearn.model_selection import train_test_split

dataset_X = dataset[dataset.columns[0]].tolist()
dataset_y = dataset[dataset.columns[1]].tolist()

# train/validation/test dataset 분할
# train:val:test = 6:2:2
train_X, temp_X, train_y, temp_y = train_test_split(dataset_X, dataset_y, test_size=0.4, random_state=0)
val_X, test_X, val_y, test_y = train_test_split(temp_X, temp_y, test_size=0.5, random_state=0)

ImportError: cannot import name '_check_response_method' from 'sklearn.utils.validation' (/opt/homebrew/Caskroom/miniforge/base/envs/DL-tf/lib/python3.8/site-packages/sklearn/utils/validation.py)

In [ ]:
print(f'train set length: {len(train_X)}')
print(f'valid set length: {len(val_X)}')
print(f'test  set length: {len(test_X)}')

## Tokenize Dataset

In [ ]:
import tensorflow as tf

In [ ]:
from transformers import BertTokenizerFast

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base", max_len=max_seq_len, truncation=True, padding=True) # klue/bert

In [ ]:
train_X = tokenizer(train_X, truncation=True, padding=True)
val_X = tokenizer(val_X, truncation=True, padding=True)

In [ ]:
# tensorflow의 dataset 객체로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_X), train_y))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_X), val_y))

## Fine-tuning Model

In [ ]:
from transformers import TFBertForSequenceClassification

In [ ]:
model_name = "klue/bert-base"
num_labels = len(dataset.groupby(by=['label']))
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

In [ ]:
# tensorflow의 dataset 객체로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_X), train_y))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_X), val_y))

In [ ]:
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, from_pt=True)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

In [ ]:
callback_earlystop = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=0
)

In [ ]:
train_dataset = train_dataset.shuffle(10000)
train_dataset = train_dataset.batch(64)
val_dataset = val_dataset.shuffle(10000)
val_dataset = val_dataset.batch(64)

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
model.fit(
    train_dataset, epochs=3, batch_size=32,
    validation_data = val_dataset,
    callbacks = [callback_earlystop]
)

## Save Model

In [ ]:
MODEL_NAME = 'best_bert'
MODEL_SAVE_PATH = os.path.join("/content/drive/MyDrive", MODEL_NAME)

if os.path.exists(MODEL_SAVE_PATH):
  print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
  os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
  print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

## Preprocessing

일반대화 예시

```json
{
	"id": {
		"text": "이거 들어봐 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 요즘 듣는 것도 들어봐 음 난 좀 별론데 좋을 줄 알았는데 아쉽네 내 취향은 아닌 듯 배고프다 밥이나 먹으러 가자 그래"
	}
}
```

In [ ]:
import json
with open('./data/test.json') as f:
    test_data = json.load(f)

test_data = pd.DataFrame(test_data).T
test_data.reset_index(drop=True)

In [ ]:
test_data = pd.DataFrame.from_dict(data=test_data).T.reset_index(drop=True)

In [ ]:
test_data

In [ ]:
split_sentences(test_data.text.iloc[2])

## Modeling

## Tuning